In [29]:
import pandas as pd
import firebase_admin
import google.cloud
from firebase_admin import credentials, firestore

cred = credentials.Certificate("./key.json")
app = firebase_admin.initialize_app(cred)

ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

In [30]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [40]:
patients = pd.read_csv('patients.csv')
print(patients.columns[:24])
patients = patients.drop('Unnamed: 0', axis=1)
# patients.rename(columns={patients.columns})
patients.columns = map(str.lower, patients.columns)
patients['name'] = patients['first'].str.replace('\d+', '', regex=True) +' ' + patients['last'].str.replace('\d+', '', regex=True)
patients = patients.drop_duplicates(subset='name')
patients = patients.drop(['first', 'last'], axis=1)
patients = patients.rename(columns={'gender': 'sex'})

patients = patients.reset_index()
patients.describe()

Index(['Unnamed: 0', 'Id', 'BIRTHDATE', 'DEATHDATE', 'SSN', 'DRIVERS',
       'PASSPORT', 'PREFIX', 'FIRST', 'LAST', 'MARITAL', 'RACE', 'ETHNICITY',
       'GENDER', 'BIRTHPLACE', 'ADDRESS', 'CITY', 'STATE', 'COUNTY', 'ZIP',
       'LAT', 'LON', 'HEALTHCARE_EXPENSES', 'HEALTHCARE_COVERAGE'],
      dtype='object')


,index,zip,lat,lon,healthcare_expenses,healthcare_coverage,alanine aminotransferase [enzymatic activity/volume] in serum or plasma,albumin [mass/volume] in serum or plasma,alkaline phosphatase [enzymatic activity/volume] in serum or plasma,american house dust mite ige ab in serum,aspartate aminotransferase [enzymatic activity/volume] in serum or plasma,bilirubin.total [mass/volume] in serum or plasma,bilirubin.total [mass/volume] in urine by test strip,body height,body mass index,body weight,body mass index (bmi) [percentile] per age and gender,body temperature,calcium,carbon dioxide,cat dander ige ab in serum,chloride,cladosporium herbarum ige ab in serum,codfish ige ab in serum,common ragweed ige ab in serum,cow milk ige ab in serum,creatinine,daly,dxa [t-score] bone density,diastolic blood pressure,egg white ige ab in serum,erythrocyte distribution width [entitic volume] by automated count,erythrocytes [#/volume] in blood by automated count,estimated glomerular filtration rate,fev1/fvc,globulin [mass/volume] in serum by calculation,glomerular filtration rate/1.73 sq m.predicted,glucose,glucose [mass/volume] in urine by test strip,head occipital-frontal circumference,heart rate,hematocrit [volume fraction] of blood,hematocrit [volume fraction] of blood by automated count,hemoglobin a1c/hemoglobin.total in blood,hemoglobin [mass/volume] in blood,hemoglobin.gastrointestinal [presence] in stool by immunologic method,high density lipoprotein cholesterol,history of hospitalizations+outpatient visits,honey bee ige ab in serum,ketones [mass/volume] in urine by test strip,latex ige ab in serum,left ventricular ejection fraction,leukocytes [#/volume] in blood by automated count,low density lipoprotein cholesterol,lymph nodes with isolated tumor cells [#] in cancer specimen by light microscopy,lymph nodes with macrometastases [#] in cancer specimen by light microscopy,lymph nodes with micrometastases [#] in cancer specimen by light microscopy,mch [entitic mass] by automated count,mchc [mass/volume] by automated count,mcv [entitic volume] by automated count,mental health outpatient note,microalbumin creatinine ratio,nt-probnp,oxygen saturation in arterial blood,pain severity - 0-10 verbal numeric rating [score] - reported,peanut ige ab in serum,percentage area affected by eczema head and neck,percentage area affected by eczema lower extremitiy - bilateral,percentage area affected by eczema trunk,percentage area affected by eczema upper extremitiy - bilateral,platelet distribution width [entitic volume] in blood by automated count,platelet mean volume [entitic volume] in blood by automated count,platelets [#/volume] in blood by automated count,polyp size greatest dimension by cap cancer protocols,potassium,prostate specific ag [mass/volume] in serum or plasma,protein [mass/volume] in serum or plasma,protein [mass/volume] in urine by test strip,qaly,qols,rbc auto (bld) [#/vol],rdw - erythrocyte distribution width auto (rbc) [entitic vol],respiratory rate,shrimp ige ab in serum,size.maximum dimension in tumor,sodium,soybean ige ab in serum,specific gravity of urine by test strip,systolic blood pressure,thyrotropin [units/volume] in serum or plasma,thyroxine (t4) free [mass/volume] in serum or plasma,total cholesterol,total score [mmse],triglycerides,urea nitrogen,wbc auto (bld) [#/vol],walnut ige ab in serum,weight difference [mass difference] --pre dialysis - post dialysis,weight-for-length per age and sex,wheat ige ab in serum,white oak ige ab in serum,ph of urine by test strip
count,1170.000000,627.000000,1170.000000,1170.000000,1.170000e+03,1170.000000,201.000000,201.000000,201.000000,58.000000,201.000000,201.000000,36.000000,1170.000000,1133.000000,1170.000000,383.000000,572.000000,415.000000,415.000000,58.00000,415.000000,58.000000,58.000000,58.00000,58.000000,415.000000,1056.000000,74.000000,1170.000000,58.000000,1170.000000,1170.000000,74.000000,26.000000,201.000000,209.000000,415.000000,36.000000,179.000000,1170.000000,81.000000,

In [41]:
                                                                       
cols = ['id', 'name', 'birthdate', 'deathdate', 'race',                                                                                                              
'ethnicity', 'sex', 'birthplace', 'healthcare_expenses',                                                                                           
'healthcare_coverage', 'condition', 'allergies', 'body height',                                                                                                        
'body mass index', 'body weight', 'body temperature', 'heart rate']

In [42]:
for i in patients.columns:
    if i not in cols:
        patients = patients.drop([i], axis=1)

In [43]:

pedi_index = []
family_index=[]
for i in patients.id.keys():
    if int(patients['birthdate'][i][:4]) > 2003:
        pedi_index.append(i)
    else:
        family_index.append(i)
len(family_index)

971

In [44]:
allergy_index = patients[patients['allergies'].notna()].index
allergy_index

Int64Index([  13,   20,   22,   26,   32,   34,   40,   45,   62,   80,
            ...
            1080, 1087, 1101, 1106, 1113, 1115, 1126, 1128, 1144, 1149],
           dtype='int64', length=141)

In [45]:
patients.iloc[0,:]

id                     1d604da9-9a81-4ba9-80c2-de3375d59b40
birthdate                                        1989-05-25
deathdate                                               NaN
race                                                  white
ethnicity                                          hispanic
sex                                                       M
birthplace                 Marigot  Saint Andrew Parish  DM
healthcare_expenses                               271227.08
healthcare_coverage                                 1334.88
condition                      Chronic sinusitis (disorder)
allergies                                               NaN
body height                                           181.0
body mass index                                        20.1
body weight                                            65.8
body temperature                                        NaN
heart rate                                             87.0
name                                    

In [46]:
db = firestore.client()

def addData(s, doctor, patient_id): 
        mode_ref = db.collection(u'Professionals').document(doctor).collection(u'Patients').document(patient_id)
#         for k, v in s.iteritems():
        mode_ref.set(s.to_dict(),merge=True)
patients.loc[i, patients.iloc[i,:].notna()][1:].astype(str).describe()


count        13
unique       13
top       white
freq          1
Name: 1169, dtype: object

In [49]:
family = u'QM7V3Ud0gbOK907UD2rMnMQiGCN2'
health_insurance = u'Hn6SZn6uj4RA7k9Owk8KHu12cRH2'
pediatrician = u'yyRs0hOgEFXFJVe5ugZWystm1uh1'
allerginist = u'5yGQKp4Fk4UY0oLkc9e5ylczmYp2'


def insuranceData(prof, clients):
    for i in range(clients):
        series = patients.loc[i, patients.iloc[i,:].notna()][1:].astype(str)
        addData(series, prof, patients.loc[i, patients.iloc[i,:].notna()][0])


def profData(prof, clients):
    for i in clients:
        series = patients.loc[i, patients.iloc[i,:].notna()][1:].astype(str)
        addData(series, prof, patients.loc[i, patients.iloc[i,:].notna()][0])


In [50]:
insuranceData(health_insurance, 500)
profData(family, family_index)
profData(allerginist, allergy_index)
profData(pediatrician, pedi_index)